In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications import InceptionV3
import matplotlib.pyplot as plt

# Define directories for the dataset
train_dir = "path_to_your_train_dataset"
val_dir = "path_to_your_validation_dataset"

# ImageDataGenerator for augmenting images
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1.0 / 255)

# Load the training data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),  # InceptionV3 default size is 299, but we use 224 here to match other layers
    batch_size=32,
    class_mode='categorical',
    shuffle=True  # Shuffling is enabled for the training data
)

# Load the validation data (with shuffle=False)
val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False  # Disabling shuffling for validation data
)

# Load the InceptionV3 model with pre-trained ImageNet weights
# Set `include_top=False` to remove the final classification layers
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model to prevent updating weights during the initial training
base_model.trainable = False

# Add custom layers on top of the InceptionV3 base model
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Global average pooling to reduce dimensionality
x = Dense(512, activation='relu')(x)  # Fully connected layer
x = Dropout(0.5)(x)  # Add dropout to prevent overfitting
predictions = Dense(train_generator.num_classes, activation='softmax')(x)  # Output layer

# Define the complete model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

# Print model summary
model.summary()

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size,
    epochs=10
)

# Fine-tune the model by unfreezing the base model and training again
base_model.trainable = True
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),  # Use a low learning rate for fine-tuning
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the entire model for additional epochs
history_fine = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size,
    epochs=10  # Additional fine-tuning epochs
)

# Save the trained model
model.save("keras_Model.h5")

# Plot training & validation accuracy values
plt.plot(history.history['accuracy'] + history_fine.history['accuracy'])
plt.plot(history.history['val_accuracy'] + history_fine.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'] + history_fine.history['loss'])
plt.plot(history.history['val_loss'] + history_fine.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()
